In [ ]:
!pip install tensorflow pandas numpy scikit-learn nltk imblearn

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2025-04-29 10:49:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-04-29 10:49:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-04-29 10:49:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!wget https://github.com/thiagorainmaker77/liar_dataset/raw/master/train.tsv
!wget https://github.com/thiagorainmaker77/liar_dataset/raw/master/valid.tsv
!wget https://github.com/thiagorainmaker77/liar_dataset/raw/master/test.tsv

--2025-04-27 14:03:17--  https://github.com/thiagorainmaker77/liar_dataset/raw/master/train.tsv
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/tfs4/liar_dataset/raw/master/train.tsv [following]
--2025-04-27 14:03:18--  https://github.com/tfs4/liar_dataset/raw/master/train.tsv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tfs4/liar_dataset/master/train.tsv [following]
--2025-04-27 14:03:18--  https://raw.githubusercontent.com/tfs4/liar_dataset/master/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# import streamlit as st
import tensorflow as tf
from imblearn.over_sampling import SMOTE

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Fake news detection/train.tsv', sep='\t', header=None)
valid = pd.read_csv('/content/drive/MyDrive/Fake news detection/valid.tsv', sep='\t', header=None)
test = pd.read_csv('/content/drive/MyDrive/Fake news detection/test.tsv', sep='\t', header=None)

In [ ]:
X_train, y_train = train[2], train[1]
X_valid, y_valid = valid[2], valid[1]
X_test, y_test = test[2], test[1]

In [ ]:
def clean_text(text):
  return text.str.lower().str.replace(r'[^\w\s]', '', regex=True)
  X_train, X_valid, X_test = clean_text(X_train), clean_text(X_valid), clean_text(X_test)

In [ ]:
max_words = 15000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_valid_seq = pad_sequences(tokenizer.texts_to_sequences(X_valid), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

In [ ]:
np.save('/content/drive/MyDrive/Fake news detection/X_train_seq.npy', X_train_seq)
np.save('/content/drive/MyDrive/Fake news detection/X_test_seq.npy', X_test_seq)
np.save('/content/drive/MyDrive/Fake news detection/X_valid_seq.npy', X_valid_seq)

In [ ]:
embedding_dim = 300
embedding_matrix = np.zeros((max_words, embedding_dim))
with open('/content/glove.6B.300d.txt', encoding='utf-8') as f:
  for line in f:
    word, *vector = line.split()
    if word in tokenizer.word_index and tokenizer.word_index[word] < max_words:
      embedding_matrix[tokenizer.word_index[word]] = np.array(vector, dtype=np.float32)

In [ ]:
def binarize_labels(labels):
    return np.where(labels.isin(['true', 'mostly-true']), 1, 0)
y_train_bin = binarize_labels(y_train)
y_valid_bin = binarize_labels(y_valid)
y_test_bin = binarize_labels(y_test)

In [ ]:
np.save('/content/drive/MyDrive/Fake news detection/y_train_mc.npy', y_train_mc)
np.save('/content/drive/MyDrive/Fake news detection/y_test_mc.npy', y_test_mc)
np.save('/content/drive/MyDrive/Fake news detection/y_valid_mc.npy', y_valid_mc)

In [ ]:
# SMOTE: Average GloVe embeddings for SMOTE
def get_avg_embedding(seq, embedding_matrix):
    embeddings = np.array([embedding_matrix[idx] for idx in seq if idx < max_words and np.any(embedding_matrix[idx])])
    return np.mean(embeddings, axis=0) if len(embeddings) > 0 else np.zeros(embedding_dim)

X_train_avg = np.array([get_avg_embedding(seq, embedding_matrix) for seq in X_train_seq])
smote = SMOTE(sampling_strategy=1.0, random_state=42)  # Balance classes
X_train_smote, y_train_smote = smote.fit_resample(X_train_avg, y_train_bin)

# Reconstruct sequences for SMOTE samples
X_train_smote_seq = np.zeros((len(X_train_smote), max_len))
for i, avg_emb in enumerate(X_train_smote):
    if i < len(X_train_seq):
        X_train_smote_seq[i] = X_train_seq[i]
    else:
        X_train_smote_seq[i] = X_train_seq[np.random.randint(0, len(X_train_seq))]

# # Save SMOTE data
# np.save(base_path + 'X_train_smote_seq.npy', X_train_smote_seq)
# np.save(base_path + 'y_train_smote.npy', y_train_smote)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_smote), y=y_train_smote)
class_weights = dict(enumerate(class_weights))

In [2]:
# For RNN+LSTM

# model = Sequential([
#     Embedding(max_words, embedding_dim, weights=[embedding_matrix], trainable=False),
#     SimpleRNN(128, return_sequences=True),
#     LSTM(128),
#     Dense(64, activation="relu"),
#     Dense(1, activation='sigmoid')
# ])



# For Bidirectional LSTM

model = Sequential([
    Embedding(max_words, embedding_dim, weights=[embedding_matrix], trainable=False),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(128)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [3]:


model_json= model.to_json()
with open("/content/drive/MyDrive/Fake news detection/model.json", "w") as f:
    f.write(model_json)

NameError: name 'model' is not defined

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Fake news detection/best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(patience=7, restore_best_weights=True)
history = model.fit(
    X_train_smote_seq, y_train_smote,
    validation_data=(X_valid_seq, y_valid_bin),
    epochs=30, batch_size=64,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop]
)

Epoch 1/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - accuracy: 0.5277 - loss: 0.6906 - val_accuracy: 0.6044 - val_loss: 0.6776
Epoch 2/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.5777 - loss: 0.6750 - val_accuracy: 0.6519 - val_loss: 0.6508
Epoch 3/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.6050 - loss: 0.6641 - val_accuracy: 0.6472 - val_loss: 0.6412
Epoch 4/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.6197 - loss: 0.6548 - val_accuracy: 0.6542 - val_loss: 0.6400
Epoch 5/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.6382 - loss: 0.6369 - val_accuracy: 0.5880 - val_loss: 0.6959
Epoch 6/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.6592 - loss: 0.6126 - val_accuracy: 0.5654 - val_loss: 0.7116
Epoch 7/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.6846 - loss: 0.5883 - val_accuracy: 0.6121 - val_loss: 0.6769
Epoch 8/30
207/207 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.7179 - loss: 0.5501 - val_

In [ ]:
y_pred_bin = (model.predict(X_test_seq) > 0.5).astype(int).flatten()
y_true_bin = y_test_bin


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


In [ ]:
accuracy = accuracy_score(y_true_bin, y_pred_bin)
f1 = f1_score(y_true_bin, y_pred_bin)
precision = precision_score(y_true_bin, y_pred_bin)
recall = recall_score(y_true_bin, y_pred_bin)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

Accuracy: 0.6346
F1: 0.4534
Precision: 0.4824
Recall: 0.4276
